In [2]:
import dask.dataframe as dd
import folium
import numpy as np
import pandas as pd
from folium.plugins import MarkerCluster
from sklearn.neighbors import NearestNeighbors

# Load large dataset with Dask
file_path = 'C:/Users/momru/Desktop/NSAC-2024/0Final Work/seal level Datasets/web/data_gapped.csv'
ddf = dd.read_csv(file_path)

# Convert to pandas for quick sampling and reduce size for visualization
sampled_df = ddf.sample(frac=0.01).compute()

# Set up interactive map with folium
def create_map():
    m = folium.Map(location=[0, 0], zoom_start=2)
    marker_cluster = MarkerCluster().add_to(m)

    for _, row in sampled_df.iterrows():
        lat = row['Latitude']
        lon = row['Longitude']
        sla = row['SLA']
        year = row['Year']
        # Tooltip only shows SLA, you can format it as needed
        folium.Marker(
            location=[lat, lon],
            tooltip=f"Sea Level: {sla}",  # Shows SLA on hover
            popup=f"Year: {year}<br>SLA: {sla}"  # Popup shows Year and SLA
        ).add_to(marker_cluster)

    return m

# Machine Learning Model
def build_knn_model(df):
    coordinates = df[['Latitude', 'Longitude']].values
    sla_data = df[['SLA', 'Year']].values
    knn = NearestNeighbors(n_neighbors=1, algorithm='auto').fit(coordinates)
    return knn, sla_data

# Query data on user click (find nearest neighbor in dataset)
def query_point(latitude, longitude, knn_model, data):
    point = np.array([[latitude, longitude]])
    distance, index = knn_model.kneighbors(point)
    sla_info = data[index[0][0]]
    return sla_info

# Usage example
m = create_map()
m.save('interactive_sea_level_map.html')

# Build and use the KNN model
knn_model, sla_data = build_knn_model(sampled_df)
clicked_point_data = query_point(10.0, 20.0, knn_model, sla_data)
